# EE5907/EE5027 Programming Assignment CA1

> by: SUN Shuo A0162488U
> 
> "You may just run the code blocks all the way till the end"

## Data Processing

In [ ]:
%matplotlib inline
import scipy.io
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Load mat data
mat_data = scipy.io.loadmat('data/spamData.mat')
#print(mat_data)
x_train = mat_data['Xtrain']
y_train = mat_data['ytrain']
x_test = mat_data['Xtest']
y_test = mat_data['ytest']

#x_train = np.array([(1,0), (1,1), (0,0)]).reshape(-1,2)
#y_train = np.array([1, 1, 0]).reshape(-1,1)
#x_test = np.array([(1,0), (1,0)]).reshape(-1,2)
#y_test = np.array([1, 1]).reshape(-1,1)

# Check data shapes and types
print("X train", type(x_train), "shape:", x_train.shape, "dtype:", x_train.dtype)
print("y train", type(y_train), "shape:", y_train.shape, "dtype:", y_train.dtype)
print("X test", type(x_test), "shape:", x_test.shape, "dtype:", x_test.dtype)
print("y test", type(y_test), "shape:", y_test.shape, "dtype:", y_test.dtype)

# Binarization 
x_train_bin = (x_train > 0) * 1
x_test_bin = (x_test > 0) * 1
#print(x_train_bin)
#print(x_test_bin)

# Log Transform
x_train_log = np.log(x_train + 0.1)
x_test_log = np.log(x_test + 0.1)
#print(x_train_log)
#print(x_test_log)

## Q1. Beta-binomial Naive Bayes (24%)

In [ ]:
def beta(N, N_1, a, b):
    """
    Compute the Beta(`alpha`, `alpha`) distribution
    """
    if (N + a + b) > 0:
        return (N_1 + a)/(N + a + b)
    else:
        return 0

def featureLikelihood(X_train, Y_train, j, x_j, c, alpha):
    """
    Compute the feature likelihood term for one (x_test, y_test=c) data point
    Class: `c`, Feature: `j`: p(x_test_j| x_i_j, y_test=c)
    """
    N = (Y_train == c).sum()
    X_train_j = X_train[:, j].reshape(-1, 1)
    N_1 = (X_train_j[Y_train == c] == 1).sum()
    #print("N:", N, "N_1:", N_1)
    
    if x_j == 1:
        return beta(N, N_1, alpha, alpha)
    else:
        return 1 - beta(N, N_1, alpha, alpha)

def posteriorPredictiveDistribution(X_train, Y_train, X_test, i, c, alpha):
    """
    Compute the posterior predictive distribution of test feature
    SUM of log(p(x_test_j | x_i_j, y_test=c))
    """
    p_sum = 0
    # For its j-th feature
    for j in range(X_test.shape[1]):
        p = featureLikelihood(X_train, Y_train, j, X_test[i][j], c, alpha)
        if p > 0:
            p_sum += np.log(p)
        #print("Term(", i, ",", j, ") is:", p)
            
    return p_sum
    
def betaBinomialNaiveBayes(X_train, Y_train, X_test, alpha):
    """
    Fit a Beta Binomial Naive Bayes Classifier on the `X_train`, `Y_train` data,
    and predict the results `Y_pred` with the given `alpha`
    """
    # Class label prior lambda
    lambda_ml = (Y_train == 1).sum() / Y_train.shape[0]
    #print("lambda_ml:", lambda_ml)
    
    Y_pred = np.zeros((X_test.shape[0], 1), dtype=int)
    # For the i-th test data
    for i in range(Y_pred.shape[0]):
        P_0 = np.log(1 - lambda_ml) + posteriorPredictiveDistribution(X_train, Y_train, X_test, i, 0, alpha)
        P_1 = np.log(lambda_ml) + posteriorPredictiveDistribution(X_train, Y_train, X_test, i, 1, alpha)
        #print(P_0)
        #print(P_1)
        if P_0 < P_1:
            Y_pred[i][0] = 1
        #print(Y_pred)
        #print("y predict", type(Y_pred), "shape:", Y_pred.shape, "dtype:", Y_pred.dtype)
    
    return Y_pred

def computeErrorRate(X_train, Y_train, X_test, Y_test, alpha):
    """
    Compute the Error Rate based on the `Y_pred` result and the given ground truth `Y_test`, 
    with a given alpha values
    """
    Y_pred = betaBinomialNaiveBayes(X_train, Y_train, X_test, alpha)
    num_error = (Y_pred != Y_test).sum()
    
    return num_error/Y_test.shape[0]
    
def compareAlphas(X_train, Y_train, X_test, Y_test, alphas):
    """
    Compute the Error Rate based on the `Y_pred` result and the given ground truth `Y_test`, 
    with varying alpha values
    """
    error_rates = np.zeros((alphas.shape[0], 1))
    for i in tqdm(range(alphas.shape[0])):
        error_rates[i] = computeErrorRate(X_train, Y_train, X_test, Y_test, alphas[i])
        
    return error_rates
    

### Compute and Plot Results

In [ ]:
# Set experimenting alpha values
alphas = np.arange(0, 100.5, 0.5)
print("Plotting error rates on the training set:")
train_error_rates = compareAlphas(x_train_bin, y_train, x_train_bin, y_train, alphas)
print("Plotting error rates on the test set:")
test_error_rates = compareAlphas(x_train_bin, y_train, x_test_bin, y_test, alphas)

# Plotting
fig, ax = plt.subplots()
line1, = ax.plot(alphas, train_error_rates, label='training')
line2, = ax.plot(alphas, test_error_rates, dashes=[6, 2], label='test')

ax.set(xlabel='alpha', ylabel='error rate', title='Q1. Beta-binomial Naive Bayes')
ax.legend()
ax.grid()

fig.savefig("pics/q1.png")
plt.show()

# Print some results
print("On the training set, the error rates for α = 1, 10, 100 are respectively:", 
      train_error_rates[2], ",", train_error_rates[20], ",", train_error_rates[-1])
print("On the test set, the error rates for α = 1, 10, 100 are respectively:", 
      test_error_rates[2], ",", test_error_rates[20], ",", test_error_rates[-1])

## Q2. Gaussian Naive Bayes (24%)

In [17]:
def gaussian(x, mu, sigma_sq):
    """
    Compute the gaussian(`mu`, `sigma_sq`) distribution of `x`
    """
    if sigma_sq > 0:
        return 1/np.sqrt(2*np.pi*sigma_sq) * np.exp(-0.5*np.power((x - mu), 2)/sigma_sq)
    else:
        return 0
    
def paramMLEstimate(X_train, Y_train, c):
    """
    Compute the ML estimate of `mean` and `var` for each feature
    """
    row_idxs = []
    for i in range(Y_train.shape[0]):
        if Y_train[i][0] == c:
            row_idxs.append(i)
    X_train_c = X_train[np.array(row_idxs), :]
    #print("X_train_c:", X_train_c.shape)
    mean = np.mean(X_train_c, axis=0)
    var = np.var(X_train_c, axis=0)
    #print("Mean:", mean[0], "shape:", mean.shape)
    #print("Var:", var[0], "shape:", var.shape)
    
    return mean, var
    
def featureLikelihood(x_j, mu, sigma_sq):
    """
    Compute the feature likelihood term for one (x_test, y_test=c) data point
    Class: `c`, Feature: `j`: p(x_test_j| x_i_j, y_test=c)
    """
    
    return gaussian(x_j, mu, sigma_sq)
    
def sumFeatureLikelihood(X_test, Means, Vars, i, c):
    """
    Compute the sum of test feature likelihood:
    SUM(log(p(x_test_j | x_i_j, y_test=c)))
    """
    p_sum = 0
    for j in range(X_test.shape[1]):
        p_sum += np.log(featureLikelihood(X_test[i][j], Means[c][j], Vars[c][j]))
        
    return p_sum

def GaussianNaiveBayes(X_train, Y_train, X_test):
    """
    Fit a Beta Binomial Naive Bayes Classifier on the `X_train`, `Y_train` data,
    and predict the results `Y_pred` with the given `alpha`
    """
    # Class label prior lambda
    lambda_ml = (Y_train == 1).sum() / Y_train.shape[0]
    #print("lambda_ml:", lambda_ml)
    Means = np.zeros((2, X_train.shape[1]))
    Vars = np.zeros((2, X_train.shape[1]))
    for i in range(2):
        Means[i], Vars[i] = paramMLEstimate(X_train, Y_train, i)
    #print("Means:", Means, "shape:", Means.shape)
    #print("Vars:", Vars, "shape:", Vars.shape)
    
    Y_pred = np.zeros((X_test.shape[0], 1), dtype=int)
    # For the i-th test data
    for i in range(Y_pred.shape[0]):
        P_0 = np.log(1 - lambda_ml) + sumFeatureLikelihood(X_test, Means, Vars, i, 0)
        P_1 = np.log(lambda_ml) + sumFeatureLikelihood(X_test, Means, Vars, i, 1)
        #print(P_0)
        #print(P_1)
        if P_0 < P_1:
            Y_pred[i][0] = 1
            
    #print(Y_pred)
    #print("y predict", type(Y_pred), "shape:", Y_pred.shape, "dtype:", Y_pred.dtype)
    
    return Y_pred

def computeErrorRate(X_train, Y_train, X_test, Y_test):
    """
    Compute the Error Rate based on the `Y_pred` result and the given ground truth `Y_test`, 
    with a given alpha values
    """
    Y_pred = GaussianNaiveBayes(X_train, Y_train, X_test)
    num_error = (Y_pred != Y_test).sum()
    return num_error/Y_test.shape[0]

In [ ]:
# Compute the train and test error rate
train_error_rate = computeErrorRate(x_train_log, y_train, x_train_log, y_train)
test_error_rate = computeErrorRate(x_train_log, y_train, x_test_log, y_test)

# Print some results
print("On the training set, the error rate is:", train_error_rate)
print("On the test set, the error rate is:", test_error_rate)

## Q3. Logistic regression (24%)

In [18]:
def sigmoid(x):
    """
    Compute the sigmoid(`x`)
    """
    return 1/(1 + np.exp(-x))

def mu(w_, x_):
    """
    Compute the sigmoid(`-w^Tx`)
    """
    return sigmoid(np.transpose(w_).dot(x_))

def NLL(w_, X, Y):
    """
    Compute the Negative Log Likelihood, NLL(`w_`)
    """
    nll_sum = 0
    for i in range(Y.shape[0]):
        mu_i = mu(w_, X[i])
        nll_sum += Y[i]*np.log(mu_i) + (1 - Y[i])*np.log(1 - mu_i)
        
    return -nll_sum

def NLLReg(w_, X, Y, _lambda):
    """
    Compute the Negative Log Likelihood with l2 regularization, NLL_reg(`w_`)
    """
    w = np.insert(w_, 0, 0.0)
        
    return NLL(w, X, Y) + 0.5*_lambda*np.transpose(w_).dot(w_)

def hessian(w_, X):
    """
    Compute the Hessian Matrix `H`
    """
    v = []
    for i in range(X.shape[0]):
        mu_i = mu(w_, X[i])
        v.append(mu_i*(1 - mu_i))
    S = np.diag(np.array(v))
    
    return np.transpose(X).dot(S).dot(X)

def hessianReg(w_, X, _lambda):
    """
    Compute the Hessian Matrix `H`
    """
    w = np.insert(w_, 0, 0.0)
    v = np.ones(w.shape[0])
    v[0] = 0.0
    
    return hessian(w, X) + _lambda*np.diag(v)



## Q4. K-Nearest Neighbors (24%)